## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

**Reporting Services Queries**

> -   List subscription information (Query 1)
> -   Report Execution Stats (Query 2)
> -   Datasource Reports - Datasource details, report path, report name (Query 3) 
> -   Report run details(Query 4)  
> -   Report definitions (Query 5)
> -   Report data source locations (Query 6)  

> List subscription information (Query 1)   
> Subscription owner, Last runtime

In [ ]:
-- Reporting Service useful script.sql  #SQL
USE [ReportServer];  -- You may change the database name. 
GO 
SELECT  USR.UserName AS SubscriptionOwner 
      ,CAT.Name
      ,SUB.ModifiedDate 
      ,SUB.[Description] 
      ,SUB.EventType 
      ,SUB.DeliveryExtension 
      ,SUB.LastStatus 
      ,SUB.LastRunTime 
      ,SCH.NextRunTime 
      ,SCH.Name AS ScheduleName       
      ,CAT.[Path] AS ReportPath 
      ,CAT.[Description] AS ReportDescription 
FROM dbo.Subscriptions AS SUB 
     INNER JOIN dbo.Users AS USR 
         ON SUB.OwnerID = USR.UserID 
     INNER JOIN dbo.[Catalog] AS CAT 
         ON SUB.Report_OID = CAT.ItemID 
     INNER JOIN dbo.ReportSchedule AS RS 
         ON SUB.Report_OID = RS.ReportID 
            AND SUB.SubscriptionID = RS.SubscriptionID 
     INNER JOIN dbo.Schedule AS SCH 
         ON RS.ScheduleID = SCH.ScheduleID 
ORDER BY CAT.Name,SUB.LastRunTime DESC
        ,CAT.[Path];



> Report Execution Stats (Query 2)     
> execution stats, reports executed the most, time taken

In [ ]:
USE ReportServer

		SELECT COUNT(Name) AS ExecutionCount,
       Name,
       SUM(TimeDataRetrieval) AS TimeDataRetrievalSum,
       SUM(TimeProcessing) AS TimeProcessingSum,
       SUM(TimeRendering) AS TimeRenderingSum,
       SUM(ByteCount) AS ByteCountSum,
       SUM([RowCount]) AS RowCountSum
  FROM (SELECT TimeStart,
               Catalog.Type,
               Catalog.Name,
               TimeDataRetrieval,
               TimeProcessing,
               TimeRendering,
               ByteCount,
               [RowCount]
          FROM Catalog
               INNER JOIN 
               ExecutionLog
                 ON Catalog.ItemID = ExecutionLog.ReportID
         WHERE Type = 2
       ) AS RE
GROUP BY Name
ORDER BY COUNT(Name) DESC,
         Name;


> Datasource Reports (Query 3)        
> Reports from data source, path of reports and report name

In [ ]:
SELECT
	C2.Name AS Data_Source_Name,
	C.Name AS Dependent_Item_Name,
	C.Path AS Dependent_Item_Path
FROM
	ReportServer.dbo.DataSource AS DS
INNER JOIN
		ReportServer.dbo.Catalog AS C ON DS.ItemID = C.ItemID
	AND
	DS.Link IN (SELECT ItemID FROM ReportServer.dbo.Catalog WHERE Type = 5) --Type 5 identifies data sources
FULL OUTER JOIN
	ReportServer.dbo.Catalog C2 ON DS.Link = C2.ItemID
WHERE
	C2.Type = 5
ORDER BY
	C2.Name ASC,
	C.Name ASC;

> -  Report run details(Query 4)  
> -  Report details, time taken, to render/process, who ran by, row counts  

In [ ]:
USE ReportServer
GO	
SELECT TimeStart,
               Catalog.Type,
               Catalog.Name,
               TimeDataRetrieval,
               TimeProcessing,
               TimeRendering,
               ByteCount,
               [RowCount],
			   UserName
          FROM Catalog
               INNER JOIN 
               ExecutionLog
                 ON Catalog.ItemID = ExecutionLog.ReportID
		 ORDER By TimeStart DESC




> -  Report definitions (Query 5)    
> -  Report definition, show the report details easy way to see statement executed

In [ ]:
--https://www.biinsight.com/querying-ssrs-report-definition-using-t-sql/
SELECT C.NAME
       , CONVERT(NVARCHAR(MAX),CONVERT(XML,CONVERT(VARBINARY(MAX),C.CONTENT))) AS REPORTXML

FROM  REPORTSERVER.DBO.CATALOG C

WHERE  C.CONTENT IS NOT NULL

            AND  C.TYPE = 2

       AND  C.NAME LIKE '%Litigation Base Data%'

     --AND CONVERT(NVARCHAR(MAX),CONVERT(XML,CONVERT(VARBINARY(MAX),C.CONTENT))) LIKE '%DESIRED_STRING%'

> -  Report data source locations (Query 6)    
> -  Report data source details  
>  - https://dba.stackexchange.com/questions/138236/get-ssrs-datasources-from-reportserver


In [ ]:
USE ReportServer
-- Connection strings of all SSRS Shared Datasources
;WITH XMLNAMESPACES  -- XML namespace def must be the first in with clause.
    (DEFAULT 'http://schemas.microsoft.com/sqlserver/reporting/2006/03/reportdatasource'
            ,'http://schemas.microsoft.com/SQLServer/reporting/reportdesigner'
     AS rd)
,SDS AS
    (SELECT SDS.name AS SharedDsName
           ,SDS.[Path]
           ,CONVERT(xml, CONVERT(varbinary(max), content)) AS DEF
     FROM dbo.[Catalog] AS SDS
     WHERE SDS.Type = 5)     -- 5 = Shared Datasource

SELECT CON.[Path]
      ,CON.SharedDsName
      ,CON.ConnString
FROM
    (SELECT SDS.[Path]
           ,SDS.SharedDsName
           ,DSN.value('ConnectString[1]', 'varchar(150)') AS ConnString
     FROM SDS
          CROSS APPLY 
          SDS.DEF.nodes('/DataSourceDefinition') AS R(DSN)
     ) AS CON
-- Optional filter:
-- WHERE CON.ConnString LIKE '%Initial Catalog%=%TFS%'
ORDER BY CON.[Path]
        ,CON.SharedDsName;

## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)